<a href="https://colab.research.google.com/github/NUG30/homework-2-duongnghiephuy123/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [1]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [13]:
import numpy as np

#Create a feature vector from an email
def email_to_feature(dict, email):
  feature=[]
  for word in dictionary:
    if word in email:
      feature.append(1)
    else:
      feature.append(0)
  return feature



#Create training features and targets from several emails 
def emails_to_test_samples(dict, test_emails):
  training_features=[]
  training_targets=[]
  for (email,target) in test_emails:
    training_features.append(email_to_feature(dictionary,email))
    training_targets.append(target)
  return (np.array(training_features),np.array(training_targets))
(training_features,training_targets)=emails_to_test_samples(dictionary,test_emails)







 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [14]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 

#Function that calculates Naive Bayes Spam filter parameters from training features and targets
def filter(training_features,training_targets):
  (num_samples,num_features)=training_features.shape

  phii1=np.zeros(num_features)
  phii0=np.zeros(num_features)

  for i in range(0,num_features):
    num1=num0=1
    den1=den0=2
    for j in range(0,num_samples):
      if training_features[j,i]==1 and training_targets[j]==1:
        num1+=1
      if training_targets[j]==1:
        den1+=1
      if training_features[j,i]==1 and training_targets[j]==0:
        num0+=1
      if training_targets[j]==0:
        den0+=1
    phii1[i]=num1/den1
    phii0[i]=num0/den0

  num=1
  den=2+len(training_targets)

  for target in training_targets:
    if target==1:
      num+=1
  phiy1=num/den

  return (phii1,phii0,phiy1)

#Run filter function on current data to get filter parameters
(phii1,phii0,phiy1)=filter(training_features,training_targets)



#Calculate final prediction from filter parameters and input
def spam_percentage(email):
  feature=email_to_feature(dictionary,email)
  a=1
  b=1
  
  for i in range(0,len(feature)):
    if feature[i]==0:
      a*=(1-phii1[i])*phiy1
      b*=(1-phii0[i])*(1-phiy1)
    else:
      a*=phii1[i]*phiy1
      b*=phii0[i]*(1-phiy1)
  
  return a/(a+b)



Test your spam filter with the following email.


In [15]:
email="the sun is shining. buy drugs now"
print(spam_percentage(email))

0.9299429164504411



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [16]:
import string
def create_dictionary(emails):
  dictionary=[]
  table=str.maketrans("","",string.punctuation)
  for (email,target) in emails:
    words=email.split()
    for word in words:
      dictionary.append(word.translate(table))
  return dictionary
#create_dictionary(test_emails)
test_emails_2=[["God, so",0],["korra^ is coming to netflix",1]]
create_dictionary(test_emails_2)

    



['God', 'so', 'korra', 'is', 'coming', 'to', 'netflix']